In [ ]:
import torch
print(torch.cuda.is_available())  # True가 출력되어야 합니다.
print(torch.cuda.device_count())  # 사용 가능한 GPU 개수가 출력됩니다.


# 표지판 학습 코드

In [ ]:
from ultralytics import YOLO

# YOLOv8 모델 객체 생성 (기본적으로 COCO pre-trained 모델을 로드)
model = YOLO('yolov8n.pt')  # 'yolov8n.pt', 'yolov8s.pt' 등 선택 가능

# 모델 학습
model.train(data='',  # yaml 파일 경로
            epochs=50,                     # 학습 에포크 수
            imgsz=640,                     # 이미지 크기
            batch=16,
            conf=0.7,
            iou=0.65,
            device=0)                      # 배치 크기

# 학습 완료 후 결과 확인
results = model.val(data='', iou=0.7) # 데이터셋 경로

# 신호등 학습 코드

In [ ]:
from ultralytics import YOLO

# YOLOv8 모델 객체 생성
model = YOLO('yolov8s.pt')

# 모델 학습
model.train(
    data="",  # 학습 데이터셋 경로가 포함된 .yaml 파일 경로
    epochs=200,  # 학습 반복 수 (에포크)
    imgsz=(320, 640),  # 입력 이미지 크기 (최소값, 최대값) - 동적 크기 조정을 위해 사용
    batch=16,  # 배치 크기 (한 번에 학습할 이미지 개수)
    device=0,  # 사용할 GPU 디바이스 번호 (0이면 첫 번째 GPU)
    # num_workers=4,  # 데이터 로더에 사용할 CPU 스레드 수 (주석 처리됨)
    mosaic=1.0,  # 모자이크 데이터 증강의 비율 (0~1 사이, 1은 항상 모자이크 사용)
    mixup=0.2,  # MixUp 데이터 증강 비율 (0~1 사이)
    degrees=20,  # 데이터 증강 시 이미지 회전 각도 범위 (최대 ±20도)
    translate=0.2,  # 데이터 증강 시 이미지 평행 이동 범위 (0~1, 비율 기준)
    scale=0.7,  # 데이터 증강 시 이미지 스케일 조정 범위 (0.7배)
    shear=5.0,  # 데이터 증강 시 이미지 기울이기 범위 (최대 ±5도)
    perspective=0.001,  # 데이터 증강 시 원근 변환 범위 (0~1)
    hsv_h=0.02,  # HSV 색상 조정 중 Hue(색상)의 변동 범위 (0~1)
    hsv_s=0.8,  # HSV 색상 조정 중 Saturation(채도)의 변동 범위 (0~1)
    hsv_v=0.5,  # HSV 색상 조정 중 Value(명도)의 변동 범위 (0~1)
    fliplr=0.5,  # 좌우 반전 증강의 확률 (0~1, 0.5는 50% 확률로 적용)
    patience=30,  # 조기 종료(Early Stopping) 기준으로, 개선 없는 에포크 허용 수
    conf=0.6,  # 학습 중 감지된 객체를 평가할 때 사용할 Confidence threshold
    iou=0.6,  # IoU(Intersection over Union) 임계값 (0~1)
    # workers=4,  # 데이터 로더를 위한 워커 수 (주석 처리됨)
    project="./",  # 학습 결과가 저장될 프로젝트 디렉토리
    save=True  # 학습 후 체크포인트 및 결과 저장 여부
)


# 학습 완료 후 결과 확인
results = model.val(
    data="", # 학습 데이터셋 경로가 포함된 .yaml 파일 경로
    imgsz=640,
    iou=0.55,
    save_json=True,
    project="./"
)

# 학습 및 검증 결과 출력
print("Validation Results:", results)


# 카메라로 테스트

In [4]:
import cv2
from ultralytics import YOLO

# 학습된 모델 파일 경로
MODEL_PATH = "" # 자신의 모델 경로로 수정

# YOLOv8 모델 로드
model = YOLO(MODEL_PATH)

# 카메라 캡처 초기화
cap = cv2.VideoCapture(0)  # 기본 카메라 사용 (0번 장치)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

# 탐지 설정
CONFIDENCE_THRESHOLD = 0.5  # 탐지 최소 신뢰도

# 클래스 색상 정의 (랜덤)
import random
random.seed(42)
COLORS = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(len(model.names))]

# 실시간 탐지 루프
while True:
    ret, frame = cap.read()
    if not ret:
        print("카메라에서 프레임을 읽을 수 없습니다.")
        break

    # 모델을 사용하여 탐지 수행
    results = model.predict(
        source=frame, 
        conf=0.5, 
        verbose=False,
        device=0, 
        imgsz=640
    )

    # 탐지 결과를 이미지에 표시
    detections = results[0].boxes.data.cpu().numpy()  # 탐지 결과 데이터
    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection
        class_id = int(class_id)
        label = f"{model.names[class_id]} {confidence:.2f}"

        # 박스 및 라벨 표시
        color = COLORS[class_id % len(COLORS)]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 화면에 출력
    cv2.imshow("YOLOv8 Detection", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 정리
cap.release()
cv2.destroyAllWindows()
